In [1]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import List, Dict, Any, Optional
import os
from dotenv import load_dotenv
import asyncio
import json
import time
from datetime import datetime

In [2]:
%pip install -r requirements.txt

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.



INFO: pip is looking at multiple versions of opentelemetry-instrumentation-fastapi to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-instrumentation-fastapi to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/798.0 kB ? eta -:--:--
   ---------------------------------------- 798.0/798.0 kB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   --------------------------- ------------ 10.2/14.8 MB 49.1 MB/s eta 0:00:01
   ---------------------------------------- 14.8/14.8 MB 35.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.3 

In [2]:
from langchain.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


c:\Users\qkd\miniconda3\envs\conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 환경 변수 로드
load_dotenv()

app = FastAPI(title="RAG 최적화 비교 시스템", version="1.0.0")


In [4]:
# CORS 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [5]:
# 데이터 모델
class QueryRequest(BaseModel):
    query: str
    optimization_methods: List[str]
    chunk_size: int = 1000
    chunk_overlap: int = 200
    top_k: int = 5

class ComparisonResult(BaseModel):
    method: str
    response: str
    retrieved_docs: List[Dict]
    response_time: float
    relevance_score: float

class UploadResponse(BaseModel):
    message: str
    filename: str
    document_count: int


In [6]:
# 전역 변수
documents = []
vectorstore = None
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("⚠️  GOOGLE_API_KEY가 설정되지 않았습니다. .env 파일에 추가해주세요.")

# LLM 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=google_api_key,
    temperature=0.3
) if google_api_key else None

# 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

⚠️  GOOGLE_API_KEY가 설정되지 않았습니다. .env 파일에 추가해주세요.


c:\Users\qkd\miniconda3\envs\conda\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\qkd\miniconda3\envs\conda\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
